In [14]:
import subprocess
import sys
import os

# 📌 Liste des bibliothèques nécessaires
REQUIRED_PACKAGES = [
    "torch", "torchaudio", "whisper", "noisereduce",
    "pydub", "soundfile", "librosa", "webrtcvad", "ffmpeg-python",
    "silero-vad", "pyannote.audio", "tqdm","pandas"
]

# 📌 Vérification et installation des dépendances
def install_packages():
    print("\n📦 Vérification et installation des dépendances...\n")
    for package in REQUIRED_PACKAGES:
        try:
            __import__(package)
        except ImportError:
            print(f"📥 Installation de {package}...")
            subprocess.run([sys.executable, "-m", "pip", "install", package], check=True)

# 📌 Vérification et installation de FFmpeg
def install_ffmpeg():
    print("\n🔍 Vérification de FFmpeg...\n")
    try:
        subprocess.run(["ffmpeg", "-version"], stdout=subprocess.PIPE, stderr=subprocess.PIPE, check=True)
        print("✅ FFmpeg est déjà installé.")
    except FileNotFoundError:
        print("⚠️ FFmpeg non trouvé ! Installation en cours...\n")
        if sys.platform == "win32":
            subprocess.run(["choco", "install", "ffmpeg", "-y"], shell=True)
        elif sys.platform == "darwin":
            subprocess.run(["brew", "install", "ffmpeg"], shell=True)
        else:  # Linux
            subprocess.run(["sudo", "apt", "install", "ffmpeg", "-y"], shell=True)

# 📌 Exécution des installations
if __name__ == "__main__":
    install_packages()
    install_ffmpeg()
    print("\n✅ Installation terminée avec succès ! 🚀")

# 📌 Importation des bibliothèques nécessaires
try:
    from pydub import AudioSegment
    import noisereduce as nr
    import librosa
    import soundfile as sf
    import torch
    import torchaudio
    from silero_vad import get_speech_timestamps, collect_chunks
    import io
    import numpy as np
    from pyannote.audio.pipelines import SpeakerDiarization
    from pyannote.core import Segment
    from tqdm import tqdm
    import whisper
    import Implementation as imp
    import pandas as pd
    print("✅ Bibliothèques importées avec succès.")
except ImportError as e:
    print(f"❌ Erreur d'importation : {e}")
    sys.exit(1)

%load_ext autoreload
%autoreload 2



📦 Vérification et installation des dépendances...

📥 Installation de ffmpeg-python...


📥 Installation de silero-vad...

🔍 Vérification de FFmpeg...

✅ FFmpeg est déjà installé.

✅ Installation terminée avec succès ! 🚀
✅ Bibliothèques importées avec succès.
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Function

## Noise 
📌 Outils : noisereduce, pydub, DeepFilterNet
📌 Objectif : Supprimer le bruit, équilibrer le volume.

In [3]:


def clean_audio(input_audio, output_audio):
    y, sr = librosa.load(input_audio, sr=16000)
    reduced_noise = nr.reduce_noise(y=y, sr=sr)
    
    # Sauvegarde du fichier nettoyé
    sf.write(output_audio, reduced_noise, sr)

    # Normalisation du volume
    sound = AudioSegment.from_file(output_audio)
    normalized_sound = sound.apply_gain(-sound.dBFS)
    normalized_sound.export(output_audio, format="wav")



/opt/anaconda3/envs/Deep_Vision/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/opt/anaconda3/envs/Deep_Vision/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/opt/anaconda3/envs/Deep_Vision/lib/python3.9/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  F

## Filtrage pour ne garder que la voix (VAD - Voice Activity Detection) 🔊

📌 Outil : Silero VAD (plus précis que WebRTC)
📌 Objectif : Supprimer les parties sans voix (silence, musique, bruit).

In [ ]:
import torchaudio
import webrtcvad
import struct
from pydub import AudioSegment

def frame_generator(frame_duration_ms, audio, sample_rate):
    """Découpe l'audio en trames de frame_duration_ms millisecondes."""
    frame_size = int(sample_rate * (frame_duration_ms / 1000.0)) * 2  # 16-bit (2 bytes par échantillon)
    for i in range(0, len(audio), frame_size):
        yield audio[i:i + frame_size]

def extract_voice_webrtc(audio_file, output_file, aggressiveness=3):
    """
    Extrait uniquement les parties parlées d'un fichier audio en utilisant WebRTC VAD.
    
    :param audio_file: Chemin du fichier audio d'entrée (WAV uniquement)
    :param output_file: Chemin du fichier audio de sortie
    :param aggressiveness: Niveau d'agressivité du VAD (0 = le plus permissif, 3 = le plus strict)
    """
    # Charger l'audio
    waveform, sr = torchaudio.load(audio_file)
    audio = waveform.numpy().flatten()

    # Convertir en mono si nécessaire
    if waveform.shape[0] > 1:
        waveform = waveform.mean(dim=0, keepdim=True)

    # Normaliser l'audio en 16-bit PCM
    audio_int16 = (audio * 32767).astype("int16")
    audio_bytes = struct.pack('<' + ('h' * len(audio_int16)), *audio_int16)

    # Initialiser WebRTC VAD
    vad = webrtcvad.Vad(aggressiveness)
    frame_duration_ms = 30  # Taille des trames (10, 20 ou 30 ms recommandées)
    frames = list(frame_generator(frame_duration_ms, audio_bytes, sr))

    # Détecter les parties parlées
    speech_frames = [frame for frame in frames if vad.is_speech(frame, sr)]

    if not speech_frames:
        print("❌ Aucune voix détectée.")
        return

    # Fusionner les segments parlés
    speech_audio = b"".join(speech_frames)

    # Sauvegarde du fichier final
    sound = AudioSegment(data=speech_audio, sample_width=2, frame_rate=sr, channels=1)
    sound.export(output_file, format="wav")



# CSV processing

In [ ]:
# Lire le fichier CSV
annotation_df = pd.read_csv("data/HateMM_annotation.csv" , sep = ",")
display(annotation_df.head())

,video_file_name,label,hate_snippet,target
0,hate_video_1.mp4,Hate,"[['00:00:34', '00:01:34']]",Blacks
1,hate_video_2.mp4,Hate,"[['00:00:06', '00:02:06']]",Blacks
2,non_hate_video_1.mp4,Non Hate,NaN,Others
3,hate_video_3.mp4,Hate,"[['00:00:03', '00:01:40'], ['00:01:41', '00:03...",Blacks
4,non_hate_video_2.mp4,Non Hate,NaN,Blacks


In [24]:
annotation_df.shape

(1083, 4)

In [ ]:

# Function to check if audio file exists in given directories
def audio_exists(audio_name, directories):
    for directory in directories:
        if os.path.exists(os.path.join(directory, audio_name)):
            return True
    return False

# Directories to check
audio_directories = ["data/Audios/hate_audios", "data/Audios/non_hate_audios"]

# Create 'Audios' column
annotation_df['Audios_extraction'] = annotation_df['video_file_name'].apply(lambda x: audio_exists(x.replace('.mp4', '.wav'), audio_directories))


In [29]:

# Display the updated dataframe
display(annotation_df.sample(10))
annotation_df["Audios_extraction"].value_counts()

,video_file_name,label,hate_snippet,target,Audios_extraction
198,hate_video_78.mp4,Hate,"[['00:00:04', '00:03:11']]",Blacks,True
521,hate_video_207.mp4,Hate,"[['00:00:10', '00:01:28']]",Blacks,True
969,non_hate_video_577.mp4,Non Hate,NaN,['Jews'],True
462,hate_video_173.mp4,Hate,"[['00:00:01', '00:00:24']]",Blacks,True
618,non_hate_video_352.mp4,Non Hate,NaN,Muslims,True
902,non_hate_video_533.mp4,Non Hate,NaN,[],True
184,non_hate_video_110.mp4,Non Hate,NaN,Whites,True
1065,non_hate_video_639.mp4,Non Hate,NaN,['Others'],True
725,non_hate_video_416.mp4,Non Hate,NaN,Others,True
636,non_hate_video_363.mp4,Non Hate,NaN,Others,True


Audios_extraction
True     1068
False      15
Name: count, dtype: int64

# Extract all audio from the videos


In [4]:
# Extract all audio from the videos
imp.extract_all_audio("data/Videos/hate_videos" , "data/Audios/hate_audios") # Extract all hate_videos audio


##########################################
✅ Audio extrait avec succès : data/Audios/hate_audios/hate_video_1.wav


✅ Audio extrait avec succès : data/Audios/hate_audios/hate_video_10.wav
✅ Audio extrait avec succès : data/Audios/hate_audios/hate_video_100.wav
✅ Audio extrait avec succès : data/Audios/hate_audios/hate_video_101.wav
✅ Audio extrait avec succès : data/Audios/hate_audios/hate_video_102.wav
✅ Audio extrait avec succès : data/Audios/hate_audios/hate_video_103.wav
✅ Audio extrait avec succès : data/Audios/hate_audios/hate_video_104.wav
✅ Audio extrait avec succès : data/Audios/hate_audios/hate_video_105.wav
✅ Audio extrait avec succès : data/Audios/hate_audios/hate_video_106.wav
✅ Audio extrait avec succès : data/Audios/hate_audios/hate_video_107.wav
❌ Échec de l'extraction audio pour : data/Videos/hate_videos/hate_video_108.mp4
✅ Audio extrait avec succès : data/Audios/hate_audios/hate_video_109.wav
✅ Audio extrait avec succès : data/Audios/hate_audios/hate_video_11.wav
✅ Audio extrait avec succès : data/Audios/hate_audios/hate_video_110.wav
✅ Audio extrait avec succès : data/Audios/hate

In [9]:

imp.extract_audio("data/Videos/hate_videos/hate_video_17.mp4", "/data/Audios/hate_audios/hate_video_17.wav")
imp.extract_audio("data/Videos/hate_videos/hate_video_108.mp4", "/data/Audios/hate_audios/hate_video_108.wav")


❌ Échec de l'extraction audio pour : data/Videos/hate_videos/hate_video_17.mp4
❌ Échec de l'extraction audio pour : data/Videos/hate_videos/hate_video_108.mp4


In [10]:
imp.extract_all_audio("data/Videos/non_hate_videos" , "data/Audios/non_hate_audios")

##########################################
✅ Audio extrait avec succès : data/Audios/non_hate_audios/non_hate_video_1.wav
✅ Audio extrait avec succès : data/Audios/non_hate_audios/non_hate_video_10.wav
✅ Audio extrait avec succès : data/Audios/non_hate_audios/non_hate_video_100.wav
✅ Audio extrait avec succès : data/Audios/non_hate_audios/non_hate_video_101.wav
✅ Audio extrait avec succès : data/Audios/non_hate_audios/non_hate_video_102.wav
✅ Audio extrait avec succès : data/Audios/non_hate_audios/non_hate_video_103.wav
✅ Audio extrait avec succès : data/Audios/non_hate_audios/non_hate_video_104.wav
✅ Audio extrait avec succès : data/Audios/non_hate_audios/non_hate_video_105.wav
✅ Audio extrait avec succès : data/Audios/non_hate_audios/non_hate_video_106.wav
✅ Audio extrait avec succès : data/Audios/non_hate_audios/non_hate_video_107.wav
✅ Audio extrait avec succès : data/Audios/non_hate_audios/non_hate_video_108.wav
✅ Audio extrait avec succès : data/Audios/non_hate_audios/non_hate_vi

# Preprocessing audios

In [ ]:
imp.preprocess_audio("data/Audios/hate_audios/hate_video_1.wav" , "data/Audios/clean/hate_audios_clean/hate_video_1.wav")
# 



Downloading: "https://github.com/snakers4/silero-vad/zipball/master" to /home/zeus/.cache/torch/hub/master.zip


✅ Audio nettoyé et sauvegardé : data/Audios/clean/hate_audios_clean/hate_video_1.wav
🎙️ Segments parlés détectés : []


[]

# IDEE
- Threashold adaptatif (music + voix , ....) + shazam ?
- Snippet trop court (+-5 secondes ?)